In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load dataset
df = pd.read_csv("/content/synthetic_flood_data.csv")
print("✅ Dataset Loaded Successfully!")
print(df.columns)

In [ ]:
# Convert binary categorical columns ("Yes"/"No") to numerical (0/1)
binary_cols = ["Road Flooding Status"]
for col in binary_cols:
    df[col] = df[col].map({"No": 0, "Yes": 1})

In [ ]:
# One-hot encode categorical columns (handling missing columns dynamically)
categorical_cols = ["Flood Severity", "Land Use Type"]
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)  # Avoid dummy variable trap

In [ ]:

# Separate features and target
X = df.drop(columns=["Flood Occurrence (1/0)"])
y = df["Flood Occurrence (1/0)"]

# Normalize data using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)
print("✅ Data Resampled using SMOTE!")

# Reshape input for LSTM (samples, timesteps, features)
X_resampled = X_resampled.reshape(X_resampled.shape[0], 1, X_resampled.shape[1])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
print(f"✅ Training Size: {X_train.shape}, Test Size: {X_test.shape}")

In [ ]:
# Build LSTM Model
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True, input_shape=(1, X_train.shape[2]))),
    Dropout(0.3),
    Bidirectional(LSTM(32, return_sequences=False)),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile Model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)

# Train Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[lr_scheduler])

In [ ]:
# Evaluate Model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\n✅ Test Accuracy: {test_accuracy:.4f}")

# Make Predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Save Model
model.save("/content/lstm_flood_model.h5")
print("\n✅ Model saved successfully!")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Flood", "Flood"], yticklabels=["No Flood", "Flood"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print("\n🔹 Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Plot Accuracy & Loss Over Epochs
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy Bar Graph
ax[0].bar(range(len(history.history["accuracy"])), history.history["accuracy"], color="blue", label="Training Accuracy", alpha=0.7)
ax[0].bar(range(len(history.history["val_accuracy"])), history.history["val_accuracy"], color="orange", label="Validation Accuracy", alpha=0.7)
ax[0].set_title("Training vs Validation Accuracy")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")
ax[0].legend()

# Loss Bar Graph
ax[1].bar(range(len(history.history["loss"])), history.history["loss"], color="red", label="Training Loss", alpha=0.7)
ax[1].bar(range(len(history.history["val_loss"])), history.history["val_loss"], color="green", label="Validation Loss", alpha=0.7)
ax[1].set_title("Training vs Validation Loss")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
ax[1].legend()

plt.tight_layout()
plt.show()